<a href="https://colab.research.google.com/github/MartijnRoozendaal1/vvr-prediction/blob/main/Model_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Setup and Imports

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import random

# Set random seed for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False




2. Define Reusable Functions


In [ ]:
def load_data(feature_path, label_path, feature_size, max_frames=25):
    # Load features and labels
    features = np.load(feature_path, allow_pickle=True).item()
    labels_df = pd.read_csv(label_path)

    # Match IDs
    features_cleaned = {
        int(video_id.split('_')[0].split('-')[0]): feat for video_id, feat in features.items()
    }
    matching_ids = set(features_cleaned.keys()).intersection(set(labels_df['ID']))
    features_matched = {id_: features_cleaned[id_] for id_ in matching_ids}
    labels_matched = labels_df[labels_df['ID'].isin(matching_ids)].sort_values(by='ID')

    # Pad/truncate features
    X = np.array([
        np.pad(
            features_matched[id_][:max_frames],
            ((0, max(0, max_frames - len(features_matched[id_]))), (0, 0)),
            mode='constant'
        )[:max_frames]
        for id_ in labels_matched['ID']
    ])
    y = np.array(labels_matched['VVR_label'].apply(lambda x: 1 if x == 'High' else 0))

    return X, y




3. Split Data and Apply SMOTE

In [ ]:
def prepare_datasets(X, y, test_size=0.3, batch_size=32):
    # Train-test-validation split
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=test_size, random_state=seed)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=seed)

    # Apply SMOTE
    X_train_flat = X_train.reshape(X_train.shape[0], -1)
    sm = SMOTE(random_state=seed)
    X_train_balanced, y_train_balanced = sm.fit_resample(X_train_flat, y_train)
    X_train_balanced = X_train_balanced.reshape(-1, X_train.shape[1], X_train.shape[2])

    # Convert to PyTorch tensors
    X_tensor_train = torch.tensor(X_train_balanced, dtype=torch.float32)
    y_tensor_train = torch.tensor(y_train_balanced, dtype=torch.float32)
    X_tensor_val = torch.tensor(X_val, dtype=torch.float32)
    y_tensor_val = torch.tensor(y_val, dtype=torch.float32)

    train_loader = DataLoader(TensorDataset(X_tensor_train, y_tensor_train), batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(TensorDataset(X_tensor_val, y_tensor_val), batch_size=batch_size)

    return train_loader, val_loader



4. Define Models

In [ ]:
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.attn = nn.Linear(hidden_size, 1)

    def forward(self, rnn_output):
        weights = torch.softmax(self.attn(rnn_output), dim=1)
        return torch.sum(weights * rnn_output, dim=1)

class RNNWithAttention(nn.Module):
    def __init__(self, rnn_type, input_size, hidden_size, output_size, num_layers=2, dropout=0.3):
        super(RNNWithAttention, self).__init__()
        if rnn_type == 'GRU':
            self.rnn = nn.GRU(input_size, hidden_size, batch_first=True, num_layers=num_layers, dropout=dropout)
        elif rnn_type == 'LSTM':
            self.rnn = nn.LSTM(input_size, hidden_size, batch_first=True, num_layers=num_layers, dropout=dropout)
        else:
            raise ValueError("Invalid rnn_type. Choose either 'GRU' or 'LSTM'.")

        self.attention = Attention(hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        rnn_output, _ = self.rnn(x)
        attended_output = self.attention(rnn_output)
        return self.fc(attended_output)




5. Train Model

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=50, patience=5):
    best_val_loss = float('inf')
    patience_counter = 0

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch).squeeze()
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                outputs = model(X_batch).squeeze()
                val_loss += criterion(outputs, y_batch).item()

        avg_train_loss = train_loss / len(train_loader)
        avg_val_loss = val_loss / len(val_loader)
        print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print("Early stopping triggered.")
            break



6. Load Data

In [ ]:
# Paths to data
resnet_features_path = '/content/drive/MyDrive/ResNet152_Features.npy'
vgg_features_path = '/content/drive/MyDrive/VGG16_Features_512.npy'
labels_path = '/content/drive/MyDrive/time_points_binary_labels.csv'

# Choose features
use_features = 'ResNet'  # Options: 'ResNet' or 'VGG'

if use_features == 'ResNet':
    X, y = load_data(resnet_features_path, labels_path, feature_size=2048)
elif use_features == 'VGG':
    X, y = load_data(vgg_features_path, labels_path, feature_size=512)




7. Prepare Datasets

In [ ]:
train_loader, val_loader = prepare_datasets(X, y, batch_size=32 if use_features == 'ResNet' else 8)


8. Initialize and Train Models

In [ ]:
# Initialize GRU and LSTM models
hidden_size, output_size = 128, 1
gru_model = RNNWithAttention('GRU', input_size=X.shape[2], hidden_size=hidden_size, output_size=output_size)
lstm_model = RNNWithAttention('LSTM', input_size=X.shape[2], hidden_size=hidden_size, output_size=output_size)

gru_optimizer = optim.Adam(gru_model.parameters(), lr=0.001, weight_decay=1e-4)
lstm_optimizer = optim.Adam(lstm_model.parameters(), lr=0.001, weight_decay=1e-4)
criterion = nn.BCEWithLogitsLoss()

# Train models
print(f"Training GRU model with {use_features} features...")
train_model(gru_model, train_loader, val_loader, criterion, gru_optimizer)

print(f"Training LSTM model with {use_features} features...")
train_model(lstm_model, train_loader, val_loader, criterion, lstm_optimizer)
